# Desafio 5

Neste desafio, vamos praticar sobre redução de dimensionalidade com PCA e seleção de variáveis com RFE. Utilizaremos o _data set_ [Fifa 2019](https://www.kaggle.com/karangadiya/fifa19), contendo originalmente 89 variáveis de mais de 18 mil jogadores do _game_ FIFA 2019.

> Obs.: Por favor, não modifique o nome das funções de resposta.

## _Setup_ geral

In [1]:
from math import sqrt

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sct
import seaborn as sns
import statsmodels.api as sm
import statsmodels.stats as st
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from yellowbrick.model_selection import RFECV
from sklearn.linear_model import LinearRegression
from loguru import logger

In [2]:
# Algumas configurações para o matplotlib.
#%matplotlib inline

from IPython.core.pylabtools import figsize


figsize(12, 8)

sns.set()

In [3]:
fifa = pd.read_csv("data/fifa.csv")

In [4]:
columns_to_drop = ["Unnamed: 0", "ID", "Name", "Photo", "Nationality", "Flag",
                   "Club", "Club Logo", "Value", "Wage", "Special", "Preferred Foot",
                   "International Reputation", "Weak Foot", "Skill Moves", "Work Rate",
                   "Body Type", "Real Face", "Position", "Jersey Number", "Joined",
                   "Loaned From", "Contract Valid Until", "Height", "Weight", "LS",
                   "ST", "RS", "LW", "LF", "CF", "RF", "RW", "LAM", "CAM", "RAM", "LM",
                   "LCM", "CM", "RCM", "RM", "LWB", "LDM", "CDM", "RDM", "RWB", "LB", "LCB",
                   "CB", "RCB", "RB", "Release Clause"
]

try:
    fifa.drop(columns_to_drop, axis=1, inplace=True)
except KeyError:
    logger.warning(f"Columns already dropped")

## Inicia sua análise a partir daqui

In [5]:
fifa.dropna(inplace=True)

## Questão 1

Qual fração da variância consegue ser explicada pelo primeiro componente principal de `fifa`? Responda como um único float (entre 0 e 1) arredondado para três casas decimais.

In [28]:
def q1():
    pca = PCA(n_components=1)
    projeto = pca.fit_transform(fifa)
    evr = pca.explained_variance_ratio_

    return round(evr[0],3)

q1()

0.565

## Questão 2

Quantos componentes principais precisamos para explicar 95% da variância total? Responda como un único escalar inteiro.

In [7]:
def q2():
    pca = PCA(n_components=0.95)
    projeto = pca.fit_transform(fifa)
    
    return projeto.shape[1]

q2()

15

## Questão 3

Qual são as coordenadas (primeiro e segundo componentes principais) do ponto `x` abaixo? O vetor abaixo já está centralizado. Cuidado para __não__ centralizar o vetor novamente (por exemplo, invocando `PCA.transform()` nele). Responda como uma tupla de float arredondados para três casas decimais.

In [30]:
x = [0.87747123,  -1.24990363,  -1.3191255, -36.7341814,
     -35.55091139, -37.29814417, -28.68671182, -30.90902583,
     -42.37100061, -32.17082438, -28.86315326, -22.71193348,
     -38.36945867, -20.61407566, -22.72696734, -25.50360703,
     2.16339005, -27.96657305, -33.46004736,  -5.08943224,
     -30.21994603,   3.68803348, -36.10997302, -30.86899058,
     -22.69827634, -37.95847789, -22.40090313, -30.54859849,
     -26.64827358, -19.28162344, -34.69783578, -34.6614351,
     48.38377664,  47.60840355,  45.76793876,  44.61110193,
     49.28911284
]

(186.556, -6.592)

In [1]:
vector_x = np.array(x)
type(vector_x)
vector_x.mean()

NameError: name 'np' is not defined

In [47]:
principalComponents = pca.fit_transform(vector_x.reshape(-1, 1))

In [62]:
pca = PCA(n_components=1)
projeto = pca.fit_transform(vector_x.reshape(-1, 1))

In [63]:
print(pca)
print(pca.explained_variance_ratio_)
print(pca.components_)
print(pca.explained_variance_)

PCA(copy=True, iterated_power='auto', n_components=1, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)
[1.]
[[1.]]
[772.56058507]


In [39]:
principalComponents

array([[ 15.88558425],
       [ 13.75820939],
       [ 13.68898752],
       [-21.72606838],
       [-20.54279837],
       [-22.29003115],
       [-13.6785988 ],
       [-15.90091281],
       [-27.36288759],
       [-17.16271136],
       [-13.85504024],
       [ -7.70382046],
       [-23.36134565],
       [ -5.60596264],
       [ -7.71885432],
       [-10.49549401],
       [ 17.17150307],
       [-12.95846003],
       [-18.45193434],
       [  9.91868078],
       [-15.21183301],
       [ 18.6961465 ],
       [-21.10186   ],
       [-15.86087756],
       [ -7.69016332],
       [-22.95036487],
       [ -7.39279011],
       [-15.54048547],
       [-11.64016056],
       [ -4.27351042],
       [-19.68972276],
       [-19.65332208],
       [ 63.39188966],
       [ 62.61651657],
       [ 60.77605178],
       [ 59.61921495],
       [ 64.29722586]])

In [9]:
def q3():
    return (0.5, 0.5)

## Questão 4

Realiza RFE com estimador de regressão linear para selecionar cinco variáveis, eliminando uma a uma. Quais são as variáveis selecionadas? Responda como uma lista de nomes de variáveis.

In [10]:
fifa_x = fifa.drop('Overall',1)
fifa_y =  fifa['Overall']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(fifa_x,  fifa_y)

In [12]:
reg = LinearRegression()
reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [13]:
rfe = RFE(reg)

In [14]:
rfe.fit(X_train, y_train)

RFE(estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False),
  n_features_to_select=None, step=1, verbose=0)

In [15]:
resultado = pd.DataFrame({
    'colunas': X_train.columns,
    'bool':rfe.get_support(),
    'coeficientes': pd.Series(reg.coef_)
})

In [16]:
resultados_validos = resultado[resultado['bool'] == True]

In [17]:
cinco_resultados = resultados_validos.sort_values(by='coeficientes', ascending=False )[:5]

In [18]:
resposta = list(cinco_resultados['colunas'])

In [20]:
def q4():
    return resposta

q4()

['Potential', 'Age', 'Reactions', 'BallControl', 'GKDiving']